## Scraping module
Scraping of straits times news article by tag (food-prices) <br>
Hard-coded a bit

### Imports

In [ ]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

In [ ]:
#!pip install newspaper3k

In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

from newspaper import Article
import pandas as pd
import datetime

### Newspaper Article Summariser

In [7]:
def getSummary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    full_text = article.text
    location = full_text[0:full_text.find("-")]
    return article.summary, article.keywords, article.publish_date, location

### StraitsTimes Scraper

In [70]:
# change to ur own directory!
driver = webdriver.Chrome("C:/Users/User/Desktop/FYP/chromedriver.exe") 

def getNews():
    output = []
    
    # get news title in a page - need a for loop to get all the pages!
    all_news_per_page=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"views-row")))
    
    # get news url (i.e. news source) & news summary & keywords using the summariser module
    for i in range(len(all_news_per_page)-1):
        news_details = []
        try: 
            xpath = f'//*[@id="content"]/section/div[4]/div/div/div/div/div/article/div/div/div/div[1]/div/div/div/div[{i+1}]/div/div/a'        
            url = all_news_per_page[i].find_element_by_xpath(xpath).get_attribute('href')
            summary = getSummary(url)
            date = summary[2].strftime('%d/%m/%Y')[3:]
            news_details.append(date)
            news_details.append(summary[3])
            news_details.append(all_news_per_page[i].text)
            news_details.append(url)
            news_details.append(summary[0])
            news_details.append(", ".join(summary[1]))
            output.append(news_details)
        except: 
            continue
    return output

In [72]:
def getStraitsTimesNews(base_url):
    driver.get(base_url)

    # close ad (if any)
    try:
        iframe=driver.find_elements_by_tag_name("iframe")[3]
        driver.switch_to.frame(iframe)
        driver.find_elements_by_tag_name("iframe")
        ad = driver.switch_to.frame(0)
        driver.find_element_by_id('pclose-btn').click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()
    
    # filter the search result by date by xpath accordingly
    # within 24 hours, within one week, within 30 days, within a year, within three years
#     filter_by_date = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,"//*[contains(text(), 'within three years')]")))[0].click()

    # get news on page
    output = getNews()
    
    # check for next page
#     nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="content"]/section/div[4]/div/div/div/div/div/article/div/div/div/div[1]/div/div/nav/ul/li[2]/a')))[0]
#     if (nextpage.get_attribute('title') == 'Go to next page'):
#         nextpage.click()
# #         print('clicked next')
#         output.extend(getNews())
#     nextpage = True
#     while (nextpage):  
#         try:            
#             nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="content"]/section/div[4]/div/div/div/div/div/article/div/div/div/div[1]/div/div/nav/ul/li[3]/a')))[0]
#             if (nextpage.get_attribute('title') == 'Go to next page'):
#                 nextpage.click()
# #                 print('clicked next')
#                 output.extend(getNews())
#             else: 
#                 nextpage = False
#         except:
# #             print("no next page!")
#             nextpage = False
    return output

output = []
for i in range(0,10):
    print(i)
    url = f"https://www.straitstimes.com/tags/food-prices?page={i}"
    output.extend(getStraitsTimesNews(url))
#     print(output)

# convert to df and export as csv
df = pd.DataFrame(output, columns=['date','location','news title', 'news source(url)', 'content summary', 'keywords'])
df.to_csv(f"csv_data/food-prices-tag_straitstimes.csv")
df

0
1
2
3
4
5
6
7
8
9


,date,location,news title,news source(url),content summary,keywords
0,02/2022,GEORGE TOWN (THE STAR/ASIA NEWS NETWORK),LATEST\nSpike in cost leaves Malaysian durian ...,https://www.straitstimes.com/asia/se-asia/spik...,GEORGE TOWN (THE STAR/ASIA NEWS NETWORK) - Dur...,"price, spike, used, fertiliser, spot, increase..."
1,02/2022,JAKARTA,Spike in cost leaves Malaysian durian farmers ...,https://www.straitstimes.com/asia/se-asia/its-...,"JAKARTA - For a few months, Jakarta fried snac...","food, cooking, sumarno, feel, rupiah, shortage..."
2,02/2022,WASHINGTON (AFP),'It's like gold now': Indonesian food vendors ...,https://www.straitstimes.com/business/economy/...,"American shoppers saw prices for meats, poultr...","washington, cent, strong, savings, prices, sto..."
3,02/2022,HONG KONG (AFP),Surging beef prices hard to stomach for US sho...,https://www.straitstimes.com/asia/east-asia/sh...,HONG KONG (AFP) - Shoppers thronged Hong Kong'...,"week, vegetable, hong, kongs, covid19, curbs, ..."
4,02/2022,LONDON (AFP),Shoppers throng Hong Kong's markets after Covi...,https://www.straitstimes.com/world/europe/wors...,"LONDON (AFP) - The ""worst is still to come"" ov...","week, cent, food, soaring, inflation, rate, pr..."
...,...,...,...,...,...,...
91,05/2015,ROME (Reuters),Increase in egg supply from Malaysia leads to ...,https://www.straitstimes.com/world/africa/worl...,ROME (Reuters) - Global food prices fell in Ap...,"reading, million, food, united, world, near, f..."
92,04/2015,Bad news for lobster fans: The price of the cr...,World food prices fell to near 5-year low in A...,https://www.straitstimes.com/lifestyle/food/be...,The result is that dishes incorporating lobste...,"restaurants, seafood, fix, prices, meat, incre..."
93,04/2015,The red grouper or pomfret that you see at the...,Be prepared to pay more for your lobster fix\n...,https://www.straitstimes.com/singapore/prices-...,"Retail prices of fish have fallen in general, ...","bigger, cent, dip, hauls, prices, pomfret, fal..."
94,03/2015,With local produce accounting for less than 10...,"Prices of fish dip amid bigger hauls\nApr 13, ...",https://www.straitstimes.com/singapore/fish-su...,About 10 per cent of Giant's fish supply comes...,"cent, local, bloom, prices, oxygen, plankton, ..."
